# 用pandas預測你的人生財務曲線

In [2]:
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import random
%matplotlib inline
def asset_prediction(起始資金 ,起始年紀,
    每月薪水 ,
    每月多投入,
    每月開銷 ,
    每月房租 ,
    退休年齡 ,
    投資部位,
    投資年利率,
    買房價格,
    買房頭期款,
    買房年紀,
    房貸利率,
    貸款年數,):

    預測時段 = range(起始年紀, 100)
    每年淨額 = pd.Series(0, index=預測時段)
    每年淨額.iloc[0] = 起始資金
    每年淨額.loc[:退休年齡] += 每月薪水 * 12
    每年淨額 -= (每月開銷 + 每月房租) * 12
    
    每年淨額加=每年淨額.copy()
    每年淨額加.loc[:退休年齡]+=每月多投入*12
    
 #   def save_more(arr,k_per_month,retire):
 #       arr2=arr
 #       arr2.loc[:retire]+= (k_per_month*12)
#        return arr2
#    
 #   每年淨額2=save_more(每年淨額,10,退休年齡)
#    每年多存金額.loc[:退休年齡]+=120
    
    
    def compound_interest(arr, ratio, return_rate):
        ret = [arr.iloc[0]]
        for v in arr[1:]:
            ret.append(ret[-1] * ratio * (return_rate/100 + 1) + ret[-1] * (1 - ratio) + v)
        return pd.Series(ret, 預測時段)
    
    買房花費 = pd.Series(0, index=預測時段)
    買房花費[買房年紀] = 買房頭期款
    買房花費.loc[買房年紀:買房年紀+貸款年數-1] += (買房價格 - 買房頭期款) / 貸款年數
    
    欠款 = pd.Series(0, index=預測時段)
    欠款[買房年紀] = 買房價格
    欠款 = 欠款.cumsum()
    欠款 = 欠款 - 買房花費.cumsum()
    利息 = 欠款.shift().fillna(0) * 房貸利率 / 100

    

    房租年繳 = pd.Series(每月房租*12, index=預測時段)
    房租年繳.loc[買房年紀:] = 0
    
    每年淨額_買房 = pd.Series(0, index=預測時段)
    每年淨額_買房.iloc[0] = 起始資金
    每年淨額_買房.loc[:退休年齡] += 每月薪水 * 12
    每年淨額_買房 -= (每月開銷*12 + 房租年繳 + 利息 + 買房花費)
    

    fig = plt.figure(figsize=(8,4), dpi=400)
    ax = fig.add_axes([0,0,1,1])
    ax.plot(compound_interest(每年淨額加, 投資部位, 投資年利率), c='g', alpha = 0.5, ls='--', label=f'save {每月多投入*10}k more per month')
    ax.plot(compound_interest(每年淨額, 投資部位, 投資年利率),c='g', alpha = 0.8, lw=3, label='invest, no house')
    ax.plot(compound_interest(每年淨額_買房, 投資部位, 投資年利率),c='r',alpha=0.8, label='invest, house')
    ax.plot(每年淨額.cumsum(),c='grey', alpha=.8 , label='no invest, no house')
    ax.plot(每年淨額_買房.cumsum(),c='black', alpha = .8, label='no invest, house')
    ax.legend()

    '''  
    compound_interest(每年淨額加, 投資部位, 投資年利率).plot(c='g', alpha = 0.5, ls='--', label='10k more',figsize=(8,4))
    compound_interest(每年淨額, 投資部位, 投資年利率).plot(c='g', alpha = 0.8, lw=3, label='invest, no house')
    compound_interest(每年淨額_買房, 投資部位, 投資年利率).plot(c='r',alpha=0.8, label='invest, house')
    每年淨額.cumsum().plot(c='grey', alpha=.8 , label='no invest, no house')
    每年淨額_買房.cumsum().plot(c='black', alpha = .8, label='no invest, house')
    plt.legend()
    
    ''' 
    '''
    pd.DataFrame({
        'no invest, no house': 每年淨額.cumsum(),
        'invest, no house': compound_interest(每年淨額, 投資部位, 投資年利率),
        'no invest, house': 每年淨額_買房.cumsum(),
        'invest, house': compound_interest(每年淨額_買房, 投資部位, 投資年利率),
        
    }).plot()
    
    
    '''
    

 #   plt.style.use('ggplot')
    plt.ylim(0, None)
#    print('most', 每年淨額.max())
    print(f'每月多投資{每月多投入}萬，最終資產 = {compound_interest(每年淨額加, 投資部位, 投資年利率).iloc[-1]:.1f} 萬')
    print(f'最終資產 = {compound_interest(每年淨額, 投資部位, 投資年利率).iloc[-1]:.1f} 萬')
    print('月繳房貸', (買房價格 - 買房頭期款) / 貸款年數 / 12)
    print('每年利息', 利息.sum() / 貸款年數)
    print('')


import ipywidgets as widgets
widgets.interact(asset_prediction, 
    起始資金=widgets.FloatSlider(min=0, max=1000, step=25, value=500),
    起始年紀=widgets.IntSlider(min=0, max=100, step=1, value=32),
    每月薪水=widgets.FloatSlider(min=0, max=40, step=0.1, value=20),
    每月多投入=widgets.FloatSlider(min=0.0, max=2, step=0.1, value=0.2),
    每月開銷=widgets.FloatSlider(min=0, max=20, step=1, value=5),
    每月房租=widgets.FloatSlider(min=0, max=20, step=0.5, value=2),
    退休年齡=widgets.IntSlider(min=0, max=100, step=1, value=39),
    投資部位=widgets.FloatSlider(min=0, max=1, step=0.1, value=1),
    投資年利率=widgets.FloatSlider(min=0, max=20, step=0.5, value=4),
    買房價格=widgets.IntSlider(min=100, max=5000, step=50, value=3000),
    買房頭期款=widgets.IntSlider(min=100, max=2000, step=50, value=100),
    買房年紀=widgets.IntSlider(min=30, max=100, step=1, value=50),
    房貸利率=widgets.FloatSlider(min=1, max=5, step=0.1, value=2),
    貸款年數=widgets.IntSlider(min=0, max=40, step=1, value=20)
                )
    

interactive(children=(FloatSlider(value=500.0, description='起始資金', max=1000.0, step=25.0), IntSlider(value=32,…

<function __main__.asset_prediction(起始資金, 起始年紀, 每月薪水, 每月多投入, 每月開銷, 每月房租, 退休年齡, 投資部位, 投資年利率, 買房價格, 買房頭期款, 買房年紀, 房貸利率, 貸款年數)>